In [1]:
#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
require("rlist")

require("lightgbm")

#paquetes necesarios para la Bayesian Optimization
require("DiceKriging")
require("mlrMBO")

#------------------------------------------------------------------------------
options(error = function() { 
  traceback(20); 
  options(error = NULL); 
  stop("exiting after script error") 
})
#------------------------------------------------------------------------------

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,613996,32.8,1330087,71.1,1150170,61.5
Vcells,1150114,8.8,8388608,64.0,1801055,13.8


Loading required package: data.table

Loading required package: rlist

Loading required package: lightgbm

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers

Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames


Warning message:
“no DISPLAY variable so Tk is not available”


In [2]:
#Parametros del script
kexperimento  <- "comp2-exp100"
# FIN Parametros del script

ksemilla  <- 100019
kBO_iteraciones  <- 50  #iteraciones de la Optimizacion Bayesiana

In [3]:
kcrossvalidation_folds  <- 5  #En caso que se haga cross validation, se usa esta cantidad de folds

#Hiperparametros FIJOS de  lightgbm
param_lgb_basicos  <- list( 
   boosting= "gbdt",          #puede ir  dart  , ni pruebe random_forest
   objective= "binary",
   metric= "custom",
   first_metric_only= TRUE,
   boost_from_average= TRUE,
   feature_pre_filter= FALSE,
   force_row_wise= TRUE,      #para que los alumnos no se atemoricen con tantos warning
   verbosity= -100,
   max_depth=  -1,            # -1 significa no limitar,  por ahora lo dejo fijo
   min_gain_to_split= 0.0,    #por ahora, lo dejo fijo
   lambda_l1= 0.0,            #por ahora, lo dejo fijo
   lambda_l2= 0.0,            #por ahora, lo dejo fijo
   max_bin= 31,               #por ahora, lo dejo fijo
   num_iterations= 9999,      #un numero muy grande, lo limita early_stopping_rounds

   bagging_fraction= 1.0,     #por ahora, lo dejo fijo
   pos_bagging_fraction= 1.0, #por ahora, lo dejo fijo
   neg_bagging_fraction= 1.0, #por ahora, lo dejo fijo

   drop_rate=  0.1,           #solo se activa en  dart
   max_drop= 50,              #solo se activa en  dart
   skip_drop= 0.5,            #solo se activa en  dart

   extra_trees= FALSE,

   seed=  ksemilla
   )

In [4]:
#Aqui se cargan los hiperparametros que se optimizan en la Bayesian Optimization
hs <- makeParamSet( 
         makeNumericParam("learning_rate",    lower=    0.005, upper=    0.3),
         makeNumericParam("feature_fraction", lower=    0.2  , upper=    1.0),
         makeIntegerParam("min_data_in_leaf", lower=    0L   , upper=  8000L),
         makeIntegerParam("num_leaves",       lower=   16L   , upper=  2048L)
        )

In [5]:
#------------------------------------------------------------------------------
#graba a un archivo los componentes de lista
#para el primer registro, escribe antes los titulos

exp_log  <- function( reg, arch=NA, folder="./exp/", ext=".txt", verbose=TRUE )
{
  archivo  <- arch
  if( is.na(arch) )  archivo  <- paste0(  folder, substitute( reg), ext )

  if( !file.exists( archivo ) )  #Escribo los titulos
  {
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  linea  <- paste0( format(Sys.time(), "%Y%m%d %H%M%S"),  "\t",     #la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  #grabo al archivo

  if( verbose )  cat( linea )   #imprimo por pantalla
}

#------------------------------------------------------------------------------

vprob_optima  <- c()

fganancia_lgbm_meseta  <- function( probs, datos) 
{
  vlabels  <- get_field(datos, "label")

  tbl  <- as.data.table( list( "prob"= probs, 
                               "gan" = ifelse( vlabels==1 , 78000, -2000  ) ) )

  setorder( tbl, -prob )
  tbl[ , posicion := .I ]
  tbl[ , gan_acum :=  cumsum( gan ) ]

  gan  <-  tbl[ , max(gan_acum) ]

  pos  <- which.max(  tbl[ , gan_acum ] ) 
  vprob_optima  <<- c( vprob_optima, tbl[ pos, prob ] )

  rm( tbl )
  return( list( "name"= "ganancia", 
                "value"=  gan,
                "higher_better"= TRUE ) )
}
#------------------------------------------------------------------------------

EstimarGanancia_lightgbm  <- function( x )
{
  gc()
  GLOBAL_iteracion  <<- GLOBAL_iteracion + 1

  param_completo  <- c( param_lgb_basicos,  x )

  param_completo$early_stopping_rounds  <- as.integer(200 + 4/param_completo$learning_rate )

  vprob_optima  <<- c()
  set.seed( param_completo$seed )
  modelo_train  <- lgb.train( data= dtrain,
                              valids= list( valid= dvalidate ),
                              eval=   fganancia_lgbm_meseta,
                              param=  param_completo,
                              verbose= -100 )

  prob_corte  <- vprob_optima[ modelo_train$best_iter ]

  #aplico el modelo a testing y calculo la ganancia
  prediccion  <- predict( modelo_train, 
                          data.matrix( dataset_test[ , campos_buenos, with=FALSE]) )

  tbl  <- dataset_test[ , list(clase_ternaria) ]
  tbl[ , prob := prediccion ]
  ganancia_test  <- tbl[ prob >= prob_corte, 
                         sum( ifelse(clase_ternaria=="BAJA+2", 78000, -2000 ) )]

  cantidad_test_normalizada  <- tbl[ prob >= prob_corte, .N ]

  rm( tbl )
  gc()

  ganancia_test_normalizada  <- ganancia_test


  #voy grabando las mejores column importance
  if( ganancia_test_normalizada >  GLOBAL_ganancia )
  {
    GLOBAL_ganancia  <<- ganancia_test_normalizada
    tb_importancia    <- as.data.table( lgb.importance( modelo_train ) )

    fwrite( tb_importancia,
            file= paste0( "impo_", GLOBAL_iteracion, ".txt" ),
            sep= "\t" )

    rm( tb_importancia )
  }


  #logueo final
  ds  <- list( "cols"= ncol(dtrain),  "rows"= nrow(dtrain) )
  xx  <- c( ds, copy(param_completo) )

  xx$early_stopping_rounds  <- NULL
  xx$num_iterations  <- modelo_train$best_iter
  xx$prob_corte  <- prob_corte
  xx$estimulos  <- cantidad_test_normalizada
  xx$ganancia  <- ganancia_test_normalizada
  xx$iteracion_bayesiana  <- GLOBAL_iteracion

  exp_log( xx,  arch= "BO_log.txt" )

  return( ganancia_test_normalizada )
}
#------------------------------------------------------------------------------
#esta es la funcion mas mistica de toda la asignatura
# sera explicada en  Laboratorio de Implementacion III

vprob_optima  <- c()
vpos_optima   <- c()

fganancia_lgbm_mesetaCV  <- function( probs, datos) 
{
  vlabels  <- get_field(datos, "label")
  vpesos   <- get_field(datos, "weight")

  tbl  <- as.data.table( list( "prob"= probs, 
                               "gan" = ifelse( vlabels==1 & vpesos > 1,
                                               78000,
                                               -2000  ) ) )

  setorder( tbl, -prob )
  tbl[ , posicion := .I ]
  tbl[ , gan_acum :=  cumsum( gan ) ]

  gan  <-  tbl[ , max(gan_acum) ]

  pos  <- which.max(  tbl[ , gan_acum ] ) 
  vpos_optima   <<- c( vpos_optima, pos )
  vprob_optima  <<- c( vprob_optima, tbl[ pos, prob ] )

  rm( tbl )
  return( list( "name"= "ganancia", 
                "value"=  gan,
                "higher_better"= TRUE ) )
}
#------------------------------------------------------------------------------

EstimarGanancia_lightgbmCV  <- function( x )
{
  gc()
  GLOBAL_iteracion  <<- GLOBAL_iteracion + 1

  param_completo  <- c(param_lgb_basicos,  x )

  param_completo$early_stopping_rounds  <- as.integer(200 + 4/param_completo$learning_rate )

  vprob_optima  <<- c()
  vpos_optima   <<- c()

  set.seed( param_completo$seed )
  modelocv  <- lgb.cv( data= dtrain,
                       eval=   fganancia_lgbm_mesetaCV,
                       param=  param_completo,
                       stratified= TRUE,                   #sobre el cross validation
                       nfold= kcrossvalidation_folds,
                       verbose= -100 )

  desde  <- (modelocv$best_iter-1)*kcrossvalidation_folds + 1
  hasta  <- desde + kcrossvalidation_folds -1

  prob_corte            <-  mean( vprob_optima[ desde:hasta ] )
  cantidad_normalizada  <-  mean( vpos_optima[ desde:hasta ] ) * kcrossvalidation_folds

  ganancia  <- unlist(modelocv$record_evals$valid$ganancia$eval)[ modelocv$best_iter ]
  ganancia_normalizada  <- ganancia * kcrossvalidation_folds


  #voy grabando las mejores column importance
  if( ganancia_normalizada >  GLOBAL_ganancia )
  {
    GLOBAL_ganancia  <<- ganancia_normalizada

    param_impo <-  copy( param_completo )
    param_impo$early_stopping_rounds  <- 0
    param_impo$num_iterations  <- modelocv$best_iter

    modelo  <- lgb.train( data= dtrain,
                       param=  param_impo,
                       verbose= -100 )

    tb_importancia    <- as.data.table( lgb.importance( modelo ) )

    fwrite( tb_importancia,
            file= paste0( "impo_", GLOBAL_iteracion, ".txt" ),
            sep= "\t" )
    
    rm( tb_importancia )
  }


  #logueo final
  ds  <- list( "cols"= ncol(dtrain),  "rows"= nrow(dtrain) )
  xx  <- c( ds, copy(param_completo) )

  xx$early_stopping_rounds  <- NULL
  xx$num_iterations  <- modelocv$best_iter
  xx$prob_corte  <-  prob_corte
  xx$estimulos   <-  cantidad_normalizada
  xx$ganancia  <- ganancia_normalizada
  xx$iteracion_bayesiana  <- GLOBAL_iteracion

  exp_log( xx,  arch= "BO_log.txt" )

  return( ganancia_normalizada )
}


In [ ]:
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------
#Aqui empieza el programa

setwd("~/buckets/b1/")
kexp_input  <- kexperimento
#cargo el dataset donde voy a entrenar
#esta en la carpeta del exp_input y siempre se llama  dataset_training.csv.gz
dataset_input  <- paste0( "./exp/", kexp_input, "/dataset_training.csv.gz" )
dataset  <- fread( dataset_input )

#Verificaciones
if( ! ("fold_train"    %in% colnames(dataset) ) ) stop("Error, el dataset no tiene el campo fold_train \n")
if( ! ("fold_validate" %in% colnames(dataset) ) ) stop("Error, el dataset no tiene el campo fold_validate \n")
if( ! ("fold_test"     %in% colnames(dataset) ) ) stop("Error, el dataset no tiene el campo fold_test  \n")
if( dataset[ fold_train==1, .N ] == 0 ) stop("Error, en el dataset no hay fold_train==1 \n")

#creo la carpeta donde va el experimento
dir.create( paste0( "./exp/", kexperimento, "/"), showWarnings = FALSE )
setwd(paste0( "./exp/", kexperimento, "/"))   #Establezco el Working Directory DEL EXPERIMENTO

cat( kexp_input,
     file= "TrainingStrategy.txt",
     append= FALSE )

#defino la clase binaria clase01
dataset[  , clase01 := ifelse( clase_ternaria=="CONTINUA", 0L, 1L ) ]


#los campos que se pueden utilizar para la prediccion
campos_buenos  <- setdiff( copy(colnames( dataset )), c( "clase01", "clase_ternaria", "fold_train", "fold_validate", "fold_test" ) )

#la particion de train siempre va
dtrain  <- lgb.Dataset( data=    data.matrix( dataset[ fold_train==1, campos_buenos, with=FALSE] ),
                        label=   dataset[ fold_train==1, clase01 ],
                        weight=  dataset[ fold_train==1, ifelse( clase_ternaria == "BAJA+2", 1.0000001, 1.0) ],
                        free_raw_data= FALSE
                      )


kvalidate  <- FALSE
ktest  <- FALSE
kcrossvalidation  <- TRUE

#Si hay que hacer validacion
if( dataset[ fold_train==0 & fold_test==0 & fold_validate==1, .N ] > 0 )
{
  kcrossvalidation  <- FALSE
  kvalidate  <- TRUE
  dvalidate  <- lgb.Dataset( data=  data.matrix( dataset[ fold_validate==1, campos_buenos, with=FALSE] ),
                             label= dataset[ fold_validate==1, clase01 ],
                             free_raw_data= FALSE  )

}

#Si hay que hacer testing
if( dataset[ fold_train==0 & fold_validate==0 & fold_test==1, .N ] > 0 )
{
  ktest  <- TRUE
  kcrossvalidation  <- FALSE
  dataset_test  <- dataset[ fold_test== 1 ]
}


#Si hay testing, sin validation,  STOP !!
if( kvalidate== FALSE & ktest== TRUE ) stop("Error, si hay testing, debe haber validation \n") 


rm( dataset )
gc()


#si ya existe el archivo log, traigo hasta donde procese
if( file.exists( "BO_log.txt" ) )
{
  tabla_log  <- fread( "BO_log.txt" )
  GLOBAL_iteracion  <- nrow( tabla_log )
  GLOBAL_ganancia   <- tabla_log[ , max(ganancia) ]
  rm(tabla_log)
} else  {
  GLOBAL_iteracion  <- 0
  GLOBAL_ganancia   <- -Inf
}


#Aqui comienza la configuracion de mlrMBO

#deobo hacer cross validation o  Train/Validate/Test
if( kcrossvalidation ) {
  funcion_optimizar  <- EstimarGanancia_lightgbmCV
} else {
  funcion_optimizar  <- EstimarGanancia_lightgbm
}


configureMlr( show.learner.output= FALSE)

#configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
#por favor, no desesperarse por lo complejo
obj.fun  <- makeSingleObjectiveFunction(
              fn=       funcion_optimizar, #la funcion que voy a maximizar
              minimize= FALSE,   #estoy Maximizando la ganancia
              noisy=    TRUE,
              par.set=  hs,     #definido al comienzo del programa
              has.simple.signature = FALSE   #paso los parametros en una lista
             )

#archivo donde se graba y cada cuantos segundos
ctrl  <- makeMBOControl( save.on.disk.at.time= 600,  
                         save.file.path=       "bayesiana.RDATA" )
                         
ctrl  <- setMBOControlTermination( ctrl, 
                                   iters= kBO_iteraciones )   #cantidad de iteraciones
                                   
ctrl  <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI() )

#establezco la funcion que busca el maximo
surr.km  <- makeLearner("regr.km",
                        predict.type= "se",
                        covtype= "matern3_2",
                        control= list(trace= TRUE) )



#Aqui inicio la optimizacion bayesiana
if( !file.exists( "bayesiana.RDATA" ) ) {

  run  <- mbo(obj.fun, learner= surr.km, control= ctrl)

} else {
  #si ya existe el archivo RDATA, debo continuar desde el punto hasta donde llegue
  #  usado para cuando se corta la virtual machine
  run  <- mboContinue( "bayesiana.RDATA" )   #retomo en caso que ya exista
}

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2230196,119.2,4427470,236.5,2787660,148.9
Vcells,122564028,935.1,329872562,2516.8,274800398,2096.6


Computing y column(s) for design. Not provided.



20221016 203528	154	480355	gbdt	binary	custom	TRUE	TRUE	FALSE	TRUE	-100	-1	0	0	0	31	655	1	1	1	0.1	50	0.5	FALSE	100019	0.191289524449094	0.367606929480098	6483	491	0.00252360919360988	12954	21612000	1
20221016 203728	154	480355	gbdt	binary	custom	TRUE	TRUE	FALSE	TRUE	-100	-1	0	0	0	31	834	1	1	1	0.1	50	0.5	FALSE	100019	0.0702372890371043	0.841137736616656	5897	1297	0.0113646314457862	10647	23746000	2
20221016 203854	154	480355	gbdt	binary	custom	TRUE	TRUE	FALSE	TRUE	-100	-1	0	0	0	31	677	1	1	1	0.1	50	0.5	FALSE	100019	0.245102375623464	0.645683627517428	2692	822	4.95517360450822e-06	12212	21416000	3
20221016 203941	154	480355	gbdt	binary	custom	TRUE	TRUE	FALSE	TRUE	-100	-1	0	0	0	31	283	1	1	1	0.1	50	0.5	FALSE	100019	0.236582367759402	0.339633598166984	4569	699	0.00588567230622915	12215	22370000	4
20221016 204101	154	480355	gbdt	binary	custom	TRUE	TRUE	FALSE	TRUE	-100	-1	0	0	0	31	397	1	1	1	0.1	50	0.5	FALSE	100019	0.120760421855084	0.882468094816431	929	2044	4.37606073706556e-05	11934	22372000

[mbo] 0: learning_rate=0.191; feature_fraction=0.368; min_data_in_leaf=6483; num_leaves=491 : y = 2.16e+07 : 109.6 secs : initdesign

[mbo] 0: learning_rate=0.0702; feature_fraction=0.841; min_data_in_leaf=5897; num_leaves=1297 : y = 2.37e+07 : 119.8 secs : initdesign

[mbo] 0: learning_rate=0.245; feature_fraction=0.646; min_data_in_leaf=2692; num_leaves=822 : y = 2.14e+07 : 85.7 secs : initdesign

[mbo] 0: learning_rate=0.237; feature_fraction=0.34; min_data_in_leaf=4569; num_leaves=699 : y = 2.24e+07 : 47.1 secs : initdesign

[mbo] 0: learning_rate=0.121; feature_fraction=0.882; min_data_in_leaf=929; num_leaves=2044 : y = 2.24e+07 : 80.4 secs : initdesign

[mbo] 0: learning_rate=0.0271; feature_fraction=0.988; min_data_in_leaf=1944; num_leaves=978 : y = 2.48e+07 : 425.5 secs : initdesign

[mbo] 0: learning_rate=0.222; feature_fraction=0.689; min_data_in_leaf=6576; num_leaves=1639 : y = 2.14e+07 : 63.8 secs : initdesign

[mbo] 0: learning_rate=0.0599; feature_fraction=0.767; min_data

20221016 210445	154	480355	gbdt	binary	custom	TRUE	TRUE	FALSE	TRUE	-100	-1	0	0	0	31	558	1	1	1	0.1	50	0.5	FALSE	100019	0.0395506724653946	0.999979737669564	3452	1487	0.0117143302990324	13629	24022000	17


...

[mbo] 42: learning_rate=0.00526; feature_fraction=0.68; min_data_in_leaf=2255; num_leaves=177 : y = 2.47e+07 : 537.3 secs : infill_ei



20221017 021411	154	480355	gbdt	binary	custom	TRUE	TRUE	FALSE	TRUE	-100	-1	0	0	0	31	7535	1	1	1	0.1	50	0.5	FALSE	100019	0.00500985526437458	0.49454315380347	2877	559	0.0066917654489758	11643	24714000	59


[mbo] 43: learning_rate=0.00501; feature_fraction=0.495; min_data_in_leaf=2877; num_leaves=559 : y = 2.47e+07 : 987.7 secs : infill_ei

Saved the current state after iteration 44 in the file bayesiana.RDATA.



20221017 021754	154	480355	gbdt	binary	custom	TRUE	TRUE	FALSE	TRUE	-100	-1	0	0	0	31	1868	1	1	1	0.1	50	0.5	FALSE	100019	0.0148891063082458	0.682814297389024	2865	181	0.00875065711134628	12599	24482000	60


[mbo] 44: learning_rate=0.0149; feature_fraction=0.683; min_data_in_leaf=2865; num_leaves=181 : y = 2.45e+07 : 221.3 secs : infill_ei

